# Bài 7: Pandas (part 5)

In [1]:
import pandas as pd
import numpy as np

## 1. Warnings

In [2]:
# Tạo sample data
df = pd.DataFrame({
    "id": range(1, 6),
    "score": ["A", "B", "B", "A", "F"],
})

In [3]:
# View
df

,id,score
0,1,A
1,2,B
2,3,B
3,4,A
4,5,F


In [4]:
# Tạo copy
df2 = df.copy()
df2

,id,score
0,1,A
1,2,B
2,3,B
3,4,A
4,5,F


## 1.1. Warning caused by "chained assigment"

#### Wrong way

In [5]:
# 1. Lọc ra sinh viên điểm A
df2.loc[df2["score"] == "A", :]

,id,score
0,1,A
3,4,A


In [6]:
# 2. Lọc ra sinh viên điểm A, truy xuất vào cột điểm
df2.loc[df2["score"] == "A", :]["score"]

0    A
3    A
Name: score, dtype: object

In [7]:
# 3. Lọc ra sinh viên điểm A, truy xuất vào cột điểm, sửa lại thành điểm C
# Chú ý sẽ phát sinh warning
df2.loc[df2["score"] == "A", :]["score"] = "C"

<ipython-input-7-7c80303fe73e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.loc[df2["score"] == "A", :]["score"] = "C"


In [8]:
# 4. Kiểm tra lại xem df2 có thực sự thay đổi
# Kết quả: không thay đổi. Why?
df2

,id,score
0,1,A
1,2,B
2,3,B
3,4,A
4,5,F


#### Giải thích: 

- Warning: `A value is trying to be set on a copy of a slice from a DataFrame.`
- Đây không phải là lỗi (Error) mà là cảnh báo (Warning) rằng code có thể có vấn đề (VD: behave theo cách ko dự tính)
- Warning trên có tên là: `SettingWithCopy`, tức mình cố gán giá trị vào một bảng copy của data frame
- Process diễn ra như sau:
    1. `df2.loc[df2["score"] == "A", :]` return 1 view đến những dòng có `score == "A"` của `df2`. Nếu muốn thay đổi `df2` qua view này thì OK không vấn đề gì.
    1. Tuy nhiên hành động tiếp theo `df2.loc[df2["score"] == "A", :]["score"]` truy xuất đến cột score của view nói ở trên, lúc này cột `score` được trả về dưới dạng series, nhưng không còn liên quan gì đến `df2` nữa. Cái này gọi là "chained indexing" - index lần 1 dùng `.loc` để trả về view, chain tiếp lần 2 dùng `["score"]` để truy xuất vào cột `score`
    1. Nếu chỉ để đọc data, ko modify thì sẽ không có vấn đề gì
    1. Tuy nhiên nếu muốn thay đổi data thì sẽ không được, như ở VD trên, và 1 warning được Pandas in ra để báo cho LTV biết là LTV đang set value (gán) trên một bản copy từ `df2`, chứ ko phải trên `df2`. Vì vậy nếu ý định là thay đổi `df2` bằng cách này thì sẽ không thực hiện đc.

#### Correct way

In [9]:
# Truy xuất toàn bộ trong .loc và thực hiện gán
# Kết quả: No warning
df2.loc[df2["score"] == "A", "score"] = "C"

In [10]:
# Kiểm tra lại df2
# Đã được update
df2

,id,score
0,1,C
1,2,B
2,3,B
3,4,C
4,5,F


## 1.2. Warning caused by "hidden chaining"

In [11]:
# Tạo lại df2 từ df
df2 = df.copy()
df2

,id,score
0,1,A
1,2,B
2,3,B
3,4,A
4,5,F


In [12]:
# 1. Subset những dòng có điểm là A, gán vào df3
df3 = df2.loc[df2["score"] == "A", :]

In [13]:
# 2. View
df3

,id,score
0,1,A
3,4,A


In [14]:
# 3. Sửa score của df3 thành C theo cách đúng ở phần 1.1
# Kết quả: vẫn có warning
df3["score"] = "C"

<ipython-input-14-43c8b24a46f5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3["score"] = "C"


In [15]:
# 4. Kiểm tra lại df3
# Kết quả: đã được thay đổi
df3

,id,score
0,1,C
3,4,C


In [16]:
# 5. Kiểm tra lại df2
# Kết quả: không có thay đổi gì
df2

,id,score
0,1,A
1,2,B
2,3,B
3,4,A
4,5,F


#### Giải thích
- Khi chạy câu lệnh `df3 = df2.loc[df2["score"] == "A", :]`, df3 thực ra không tạo 1 bản copy của những dòng `score == "A"` từ df2, mà chỉ tạo 1 view đến những dòng này. Mục đích để tiết kiệm bộ nhớ, tránh tạo duplicate data.
- Do vậy `df3` thực ra không tách rời hẳn `df2`, mà được gọi là một `weak reference` đến `df2`
- Tuy nhiên khi ta cố thay đổi `df3` thông qua `df2` thì không được (vì lý do chained indexing ở trên)
- Do vậy mọi thay đổi ở `df3` sẽ báo một warning để cảnh báo LTV là những thay đổi trên `df3` chỉ có tác dụng trên `df3`, chứ không có tác dụng trên `df2`
- Để tránh hiện warning kiểu này, khi subset data nên dùng `.copy()` để `df3` độc lập hẳn với `df2`

In [17]:
# 1. Tạo df3 là bản copy độc lập
df3 = df2.loc[df2["score"] == "A", :].copy()

In [18]:
# 2. View
df3

,id,score
0,1,A
3,4,A


In [19]:
# 3. Sửa df3
# Kết quả: No warning
df3["score"] = "C"

In [20]:
# 4. View df3
# Kết quả: Đã được update
df3

,id,score
0,1,C
3,4,C


In [21]:
# 5. View df2
# Kết quả: Không bị thay đổi
df2

,id,score
0,1,A
1,2,B
2,3,B
3,4,A
4,5,F


### 1.3. Takeaway

1. Khi subset df theo một condition nào đó, rồi sửa một cột cho những dòng đó thì tất cả phải truy xuất  `.loc` và gán trong một câu lệnh. 
    - Ví dụ: lọc ra những dòng có `score == "A"` và sửa cột score cho những dòng này thành `"C"` thì làm như sau:
    - `df.loc[df["score"] == "A", "score"] = "C"`
    
1. Khi subset df dựa trên một condition nào đó và lưu ra một df khác (có thể sẽ sửa đổi sau này) thì nên dùng `.copy()`

## 2. Sum up everything

### A. Overview
- Pandas có 2 data structures chính: `Series` (1 chiều) và `DataFrame` (2 chiều)
- Mỗi cột trong `DataFrame` là một `Series`
- Cách data by hand thường dùng:
    - `s = pd.Series([1, 2, 3])`
    - `df = pd.DataFrame({"id": [1, 2], "name": ["Jane",, "Jack"]})`

In [22]:
# Example:
df = pd.read_csv("data/wide_data.csv")
df.head(2)

,date,TMAX,TMIN,TOBS
0,2018-10-01,21.1,8.9,13.9
1,2018-10-02,23.9,13.9,17.2


### B. Working with tabular data

#### B1. Load data
- `pd.read_xxx`: `pd.read_excel`, `pd.read_csv`, `pd.read_json`, ...
- Lưu ý thử các options thông dụng cho nhiều lựa chọn đọc (`?pd.read_excel`)
    - Đọc từ sheet chỉ định: `sheet_name=1`
    - Load một số cột nhất định: `usecols=["A", "B", "C"]`
    - Chỉ định kiểu dữ liệu: `dtype={"phone_number": str, "id": str}`
    - Bỏ qua N dòng đầu: `skiprows=N`
    - Load N dòng đòng: `nrows=N`
    - Có parse những cột string có dạng datetime thành datetime: `parse_dates=True`
    - ....
    - For more: google "pandas how to ___ stackoverflow"

#### B2. Inspect data quickly
- Xem N dòng đầu: `df.head(N)`
- Xem N dòng cuối: `df.tail(N)`
- Xem shape: `df.shape`
- Số dòng: `df.shape[0]`
- Số cột: `df.shape[1]`
- Data types của các cột: `df.dtypes`
- Tên cột: `df.columns.tolist()`

#### B3. Operations on columns
- Trích xuất 1 cột -> trả về một Series: `df["col_1"]`
- Trích xuất nhiều cột -> Trả về một DataFrame: `df[["col_1", "col_2"]]`
- Update một cột có sẵn: `df["col_1"] = value`
- Thêm một cột mới: `df["new_col"] = new_value`
- Đổi data type của một cột:
    - Đổi về string: `df["col_1"] = df["col_1"].astype(str)`
    - Đổi về float: `df["col_1"] = df["col_1"].astype(float)`
    - Đổi về datetime: `df["col_1"] = pd.to_datetime(df["col_1"])
    - Đổi về categorical: `df["col_1"] = pd.Categorical(df["col_1"], categories=["lv_1", "lv_2"])`
- Tính toán trên các cột:
    - Nhân 1 cột với 1000: `df["col_1"] * 1000`
    - Nhân 2 cột với nhau: `df["col_1"] * df["col_2"]`
    - Tương tự cho `+, -, /, **, ...`
    
- Thao tác trên cột `str`:
    - Upper: `df["col_1"].str.upper()`
    - Lower: `df["col_1"].str.lower()`
    - Tương tự cho các methods khác: 
        - Check bắt đầu, kết thúc: `startswith, endswith, contains`
        - Check is_XXX: `isalpha, isalnum, isupper, islower, isdecimal ...`
        - Remove khoảng trắng: `strip, lstrip, rstrip`
        - Other: `zfill, ljust, rjust, split, len`
        - ...
    - Chaining: `df["col_1"].str.strip().str.upper().str.contains("USA")`
- Thao tác trên cột `datetime`:
    - Get year: `df["col_1"].dt.year`
    - Tương tự: 
        - month, day, hour, minute, second
        - dayofyear: 1-365/366
        - week: 1-52
        - dayofweek: Monday=0, Sunday=6
        - quarter: Jan-Mar = 1, Apr-Jun = 2, etc.
        - days_in_month: 1-28/30/31
        - is_leap_year: True nếu là năm nhuận, False nếu không phải
    - Convert date sang string với format khác nhau: `df["col_1"].dt.strftime("%d/%m/%Y")`
    - Tên ngày trong tuần: `df["col_1"].dt.day_name()`
    - Cộng / trừ thêm ngày: `df["col_1"] +  pd.Timedelta('1 day')`
    - Cộng / trừ thêm business day: `df["col_1"] +  pd.offsets.BDay(1)`
    - Tạo nhanh một Series datetime: `pd.date_range("20201001", "20201015")`
    - Full docs: https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html

#### B4. Operations on rows
- Lọc các rows dựa trên condition: `df.loc[cond, :]`
- Condition thường được tạo từ các phép so sánh:
    - `==, <, >, <=, >=`
    - `.isnull()`, `.notnull()`, `.isdecimal()`
    - `.isin()`
- Kết hợp nhiều condition lại với nhau dùng `&` (and) và `|` (or)
- Khi có nhiều conditions, mỗi condition nên để trong dấu ngoặc

#### B5. Operations on both rows and columns
- Vừa filter rows, vừa chọn cột: `df.loc[cond, cols]`

#### B6. Summary statistics
- Aggregation trên 1 cột:
    - `df["col_1"].sum()`
    - Tương tự cho các methods khác: `count, mean, median, min, max, std, var...`
    - Distribution: 
        - Absolute count: `df["col_1"].value_counts()`
        - Relative proportion: `df["col_1"].value_counts(normalize=True)`
        
    - Sort:
        - Sort theo index: `.sort_index()`
        - Sort theo giá trị: `.sort_values()`
        
- Aggregation trên DataFrame:
    - `df.groupby(["col_1", "col_2"])["col_3"].sum()`
    - Tương tự cho các methods khác: `count, mean, median, min, max, std, var...`
    - Reset index nếu muốn `["col_1", "col_2"]` thành 2 cột của kết quả thay vì là index
    - Sort:
        - Sort theo index: `.sort_index()`
        - Sort theo giá trị: `.sort_values()`
    
- Describe nhanh data: `df.describe()`

#### B6. Output
- `df.to_xxx(file_name, index=False)`
- Example:
    - `df.to_excel("data/top5_countries_2020_July.xlsx", index=False)`
    - `df.to_csv("data/top5_countries_2020_July.csv", index=False)`

#### B7. Other operations
- Apply: 
    - Apply trên cột: `df["col_1"].apply(lambda x: x)`
    - Apply trên df (rowwise - across rows): `df[cols].apply(lambda col: col, axis=0)`
    - Apply trên df (columnwise - across columns): `df[cols].apply(lambda row: row, axis=1)`
    
- Chọn cột dựa vào condition:
    - Chọn các cột có dtype là số: `df.select_dtypes("number")`
    - Chọn các cột có dtype là object: `df.select_dtypes("O")`
    - Chọn cột dựa vào partial name matching:
        - Bắt đầu bởi "Ab": `df.filter(regex="^Ab")`
        - Kết thúc bởi "Ab": `df.filter(regex="Ab$")`
        - Có chứa "Ab": `df.filter(regex="Ab")`
        
- Rename columns:
    - Rename all: `df.columns = ["c1", "c2"]`
    - Rename some: `df.rename({"old_c1": "new_c1", "old_c2": "new_c2"}, inplace=True)`
    
- Cut một cột numeric thành một cột Categorial: 
    - VD:
    ```
    bins = [-np.infty, cutoff_1, cutoff_2, np.infty]
    labels = ["low", "medium", "high"]
    pd.cut(df["col_1"], bins=bins, labels=labels, right=False)
    ```
    
- Sắp xếp rows dựa theo cột:
    - `df.sort_values(["c1", "c2"], ascending=[True, False])`
    
- Lấy top rows theo một cột nào đó:
    - Top N dòng có giá trị cột `c1` nhỏ nhất: `df.nsmallest(N, "c1")`
    - Top N dòng có giá trị cột `c1` lớn nhất: `df.nlargest(N, "c1")`
    
- Reshape data:
    - Đổi hàng thành cột (transpose): `df.T`
    - Long -> Wide: `df.pivot`
    - Wide -> Long: `df.melt`
    
- Concatenate data:
    - Top-Bottom: `pd.concat([df1, df2, df3], axis=0, ignore_index=True)`
    - Side-by-side: `pd.concat([df1, df2, df3], axis=1, ignore_index=True)`
    
- Merge / join data:
    - Inner join: `pd.merge(df1, df2, on=["c1", "c2"], how="inner")`
    - Left join: `pd.merge(df1, df2, on=["c1", "c2"], how="left")`
    - Full join: `pd.merge(df1, df2, on=["c1", "c2"], how="outer")`
    
- Replace: `df.replace(to_replace=["?", "", " "], value=np.nan, inplace=True)`
- Fill NA: 
    - Fill all cols: `df.fillna(0)`
    - Fill some cols: `df.fillna({"c1": "Not available", "c2": 0})`
- Drop NA:
    - Drop rows if one col has NA: `df.dropna()`
    - Drop rows if all cols have NA: `df.dropna(how="all")`
    - Drop rows if some cols have NA: `df.dropna(subset=["c1", "c2"])`
    
- Dummy variables (one-hot encoding):
    - Get dumies: `dummies = pd.get_dummies(df["c1"], prefix="is", drop_first=True)`
    - Combine with original data: `df.join(dummies).drop(columns=["c1"])`